In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import numpy as np, pandas as pd
import os
import datetime
import csv
import time

In [2]:
#读入csv中的数据，（导入检索式）
a = []
csv_reader=csv.reader(open(r'C:\Users\rk\Desktop\UK.csv'))
for row in csv_reader:
    a.append(row)#将读入的数据赋值给列表a
#del a[0]#删除表头
c='AND DT=(Article or Review)'
for lst in a:
    lst[1]=lst[1]+' '+c
count=0

In [3]:
a[0]

['拉夫堡大学',
 'AD=((Loughborough Univ or Univ Loughborough) same sport* same England) AND DT=(Article or Review)']

In [4]:
del a[0]

In [5]:
a[0]

['利物浦约翰摩尔斯大学',
 'AD=((Liverpool John Moores Univ or Liverpool JMU) same sport* same England) AND DT=(Article or Review)']

In [6]:
a[0][1]

'AD=((Liverpool John Moores Univ or Liverpool JMU) same sport* same England) AND DT=(Article or Review)'

In [7]:
########################服务器崩溃后，从此处运行##########################################

In [8]:
browser = webdriver.Chrome('C:/Users/rk/Desktop/chromedriver.exe') #锁定浏览器

In [9]:
######################此处与下面重复，但程序刚开始运行时，及服务器崩溃后需要重新运行###################
browser.get("http://apps.webofknowledge.com/WOS_AdvancedSearch_input.do?SID=8EJqZrebXFQVSmd9PsP&product=WOS&search_mode=AdvancedSearch")

In [ ]:
#此处是程序运行出问题，无法进行，只能清除所有，重新启动后将count设置为之前的，接着运行，因为已经保存TXT，所以放心
count=294 
#再三强调，不能随意执行这一步，除非程序重启

In [ ]:
##########################程序中断后，从此处运行#####################################

print(count)
for item in a[count:]:
    browser.get("http://apps.webofknowledge.com/WOS_AdvancedSearch_input.do?SID=8EJqZrebXFQVSmd9PsP&product=WOS&search_mode=AdvancedSearch")
    try:
        if int(browser.find_element_by_class_name('historySetNum').text[2:])==200:#因为搜索历史只能保存200条，所以若达到则全选删除记录
            browser.find_element_by_xpath('/html/body/div[12]/form/table/tbody/tr[1]/th[6]/div[2]/button[1]').click()#全选
            browser.find_element_by_xpath("//input[@type='checkbox'and @name='dSet']").click()#取消第一个
            browser.find_element_by_xpath('/html/body/div[12]/form/table/tbody/tr[1]/th[6]/div[2]/button[2]/i').click()#删除
    except:
        pass
    
    browser.find_element_by_id('value(input1)').clear()#清除文本框里的内容
    browser.find_element_by_id('value(input1)').send_keys(item[1])#往文本框里输入搜索文本（输入检索式）
    browser.find_element_by_id('searchButton')#定位搜索按钮
    browser.find_element_by_id('searchButton').click()#点击搜素按钮
    
    d=browser.find_element_by_xpath('//*[@id="client_error_input_message"]').text#定位异常信息
    
    if len(d)==0:#判断检索式是否异常（如果正常运行，则不会弹出异常，那么异常信息len为零）
        v=browser.find_element_by_class_name('historyResults').text
             #如果没有异常，则定位第一个数据，因为每搜索一次，数据系列就增加一次，无法用ID定位，只能广泛定位，自动匹配第一个数据
        item.append(v)#将找到的数据拼接到原数据的后面
    else: 
        item.append('NA')#搜索异常的时候讲自定义的异常标识拼接到原数据后
        item.append('NA')
        with (open(r'C:\Users\rk\Desktop\papers.txt','a+')) as file:
            str='、'.join(item)
            file.write(str+"\n")
        count+=1
        print(item)
        print(count)
        continue
        
    #查询国际论文合作数会导致历史记录以公差为2递增，故前后设置了两次删除   
    if int(browser.find_element_by_class_name('historySetNum').text[2:])==200:#因为搜索历史只能保存200条，所以若达到则全选删除记录
        browser.find_element_by_xpath('/html/body/div[12]/form/table/tbody/tr[1]/th[6]/div[2]/button[1]').click()#全选
        browser.find_element_by_xpath("//input[@type='checkbox'and @name='dSet']").click()#取消第一个
        browser.find_element_by_xpath('/html/body/div[12]/form/table/tbody/tr[1]/th[6]/div[2]/button[2]/i').click()#删除
         #time.sleep(5)
    

#     删除历史记录，以防超过200个，这里是每查询一次就删除一次
#     browser.find_element_by_xpath('/html/body/div[12]/form/table/tbody/tr[1]/th[6]/div[2]/button[1]').click()#全选
#     browser.find_element_by_xpath("//input[@type='checkbox'and @name='dSet']").click()#取消第一个
#     browser.find_element_by_xpath('/html/body/div[12]/form/table/tbody/tr[1]/th[6]/div[2]/button[2]').click()#删除
        
        
    #获取除自引的被引频数
    browser.find_element_by_class_name('historyResults').click() 
    browser.find_element_by_xpath('//*[@id="refine_form"]/div[4]/a').click()#点击查看全部选项
    d=browser.find_element_by_xpath('//*[@id="CountryTerritory_tr"]').text
    if len(d)==0:
        browser.find_element_by_xpath('//*[@id="CountryTerritory"]/h4').click()#如果国家地区没有展开，则点击国家地区
    time.sleep(4)
    try:
        browser.find_element_by_xpath('//*[@id="CountryTerritory_tr"]/div[2]/a').click()#点击更多选项分类
        inputs=browser.find_elements_by_xpath('//*[@id="CountryTerritory_raMore_tr"]/td/table/tbody//input')#查找所有复选项
        for i in inputs:
            if i.get_attribute('type')== 'checkbox':#点击所有复选框
                i.click()
            #time.sleep(0.3)
        UK=('WALES','ENGLAND','SCOTLAND','NORTH IRELAND')#英国的四个地区
        h=0
        inputs1=browser.find_elements_by_xpath('//*[@id="CountryTerritory_raMore_tr"]/td/table/tbody//label')
        g=inputs1[0].text
        p1=g[:g.find('(')]
        p=p1.strip()
        for region in UK:#判断地区是否属于英国
            if region == p:
                h=1
                break   
        if h==1:
            for i in range(0,len(inputs1)):
                m=inputs1[i].text
                n=m[:m.find('(')]
                t=n.strip() 
                for j in range(0,len(UK)):
                    if t==UK[j]:
                        inputs1[i].click()#如果属于则再次点击复选框，以取消选项
        else:inputs[0].click()#再次点击第一个复选框
        browser.find_element_by_xpath('//*[@id="raMore"]/table/tbody/tr[6]/td/table/tbody/tr/td[1]/div/button').click()#点击精炼
        v2=browser.find_element_by_xpath('//*[@id="hitCount.top"]').text#提取国际合作论文数
        item.append(v2)
        
        #保存每条记录，浏览器，程序及服务器会崩溃，以防重新执行浪费时间
        with (open(r'C:\Users\rk\Desktop\papers.txt','a+')) as file:
            str='、'.join(item)
            file.write(str+"\n")
        count+=1
        print(item)
        print(count)
    except:
        inputs2=browser.find_elements_by_xpath('//*[@id="CountryTerritory"]/div/div/div/div/input')#查找所有复选项
        if len(inputs2)==1:
            item.append("zero")
            with (open(r'C:\Users\rk\Desktop\papers.txt','a+')) as file:
                str='、'.join(item)
                file.write(str+"\n")
            count+=1
            print(item)
            print(count)
            continue
        else:
            for inp in inputs2:
                if inp.get_attribute('type')== 'checkbox':#点击所有复选框
                    inp.click()
        UK=('WALES','ENGLAND','SCOTLAND','NORTH IRELAND')#英国的四个地区
        s=0
        inputs3=browser.find_elements_by_xpath('//*[@id="CountryTerritory_raMore_tr"]/td/table/tbody//label')
        f=inputs3[0].text
        q1=f[:g.find('(')]
        q=q1.strip()
        for region1 in UK:#判断地区是否属于英国
            if region1 == q:
                s=1
                break   
        if s==1:
            for e in range(0,len(inputs2)):
                u=inputs2[i].text
                v=u[:u.find('(')]
                z=v.strip()
                for r in range(0,len(UK)):
                    if z==UK[r]:
                        inputs2[e].click()#如果属于则再次点击复选框，以取消选项
        else:inputs2[0].click()
        browser.find_element_by_xpath('//*[@id="CountryTerritory"]/div/div/button[1]').click()#点击精炼
        v3=browser.find_element_by_xpath('//*[@id="hitCount.top"]').text#提取国际合作论文数
        item.append(v3) 
        with (open(r'C:\Users\rk\Desktop\papers.txt','a+')) as file:
            str='、'.join(item)
            file.write(str+"\n")
        count+=1
        print(item)
        print(count)
    

0
['利物浦约翰摩尔斯大学', 'AD=((Liverpool John Moores Univ or Liverpool JMU) same sport* same England) AND DT=(Article or Review)', '638', '452']
1
['伯明翰大学', 'AD=(Univ Birmingham same sport* same England) AND DT=(Article or Review)', '617', '302']
2


In [25]:
len(a[0])

8

In [26]:
for index,ite in enumerate(a):
    if(len(ite)!=8):
        print(index,ite)

26 ['China-hk', 'GC03010', '香港公开大学', 'OPEN UNIV HONG KONG SAME HONGKONG', 'AD=(OPEN UNIV HONG KONG SAME HONGKONG) AND SO=(nature or science)', 'AD=(OPEN UNIV HONG KONG SAME HONGKONG) AND DT=(Article or Review)', 'NA']


In [27]:
a[26]

['China-hk',
 'GC03010',
 '香港公开大学',
 'OPEN UNIV HONG KONG SAME HONGKONG',
 'AD=(OPEN UNIV HONG KONG SAME HONGKONG) AND SO=(nature or science)',
 'AD=(OPEN UNIV HONG KONG SAME HONGKONG) AND DT=(Article or Review)',
 'NA']

In [28]:
a[26].append("NA")

In [29]:
a[26]

['China-hk',
 'GC03010',
 '香港公开大学',
 'OPEN UNIV HONG KONG SAME HONGKONG',
 'AD=(OPEN UNIV HONG KONG SAME HONGKONG) AND SO=(nature or science)',
 'AD=(OPEN UNIV HONG KONG SAME HONGKONG) AND DT=(Article or Review)',
 'NA',
 'NA']

In [51]:
result=pd.DataFrame(np.array(a))#将pd.DataFrame(np.array(myarray))写入list
result.to_csv("C:/Users/rk/Desktop/IC.csv",index=False,encoding='ANSI')
result

,0,1,2,3,4,5
0,河北工业大学,Hebei Univ Technol same China,AD=(Hebei Univ Technol same China) AND DT=(Art...,AD=(Hebei Univ Technol same China) AND SO=(NAT...,"1,963",338
1,内蒙古大学,Inner Mongolia Univ same China not (Inner Mong...,AD=(Inner Mongolia Univ same China not (Inner ...,AD=(Inner Mongolia Univ same China not (Inner ...,"1,421",251
2,大连海事大学,Dalian Maritime Univ same China,AD=(Dalian Maritime Univ same China) AND DT=(A...,AD=(Dalian Maritime Univ same China) AND SO=(N...,"1,425",339
3,东北农业大学,NE Agr Univ same China,AD=(NE Agr Univ same China) AND DT=(Article or...,AD=(NE Agr Univ same China) AND SO=(NATUR OR S...,"2,233",347
4,海南大学,Hainan Univ same China,AD=(Hainan Univ same China) AND DT=(Article or...,AD=(Hainan Univ same China) AND SO=(NATUR OR S...,"1,309",211
5,辽宁大学,Liaoning Univ same China not (Liaoning Univ Te...,AD=(Liaoning Univ same China not (Liaoning Uni...,AD=(Liaoning Univ same China not (Liaoning Uni...,"1,123",281
6,成都中医药大学,(CHENGDU UNIV TRADIT* CHINES* MED* OR CHENGDU ...,AD=((CHENGDU UNIV TRADIT* CHINES* MED* OR CHEN...,AD=((CHENGDU UNIV TRADIT* CHINES* MED* OR CHEN...,719,82
7,广州中医药大学,GUANGZHOU UNIV CHINESE MED SAME CHINA,AD=(GUANGZHOU UNIV CHINESE MED SAME CHINA) AND...,AD=(GUANGZHOU UNIV CHINESE MED SAME CHINA) AND...,"1,100",182
8,南京林业大学,(FORESTRY UNIV NANJING OR NANJING FOREST* UNIV...,AD=((FORESTRY UNIV NANJING OR NANJING FOREST* ...,AD=((FORESTRY UNIV NANJING OR NANJING FOREST* ...,"2,219",762
9,南京中医药大学,Nanjing Univ Chinese Med SAME CHINA,AD=(Nanjing Univ Chinese Med SAME CHINA) AND D...,AD=(Nanjing Univ Chinese Med SAME CHINA) AND S...,"1,709",238
